# Setting up the environment

In [1]:
# import relevant libraries
import requests # to download html code
from bs4 import BeautifulSoup # to navigate through the html code
import pandas as pd
import numpy as np
import re
import random
import config

In [2]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials


# initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
import numpy as np

In [4]:
from IPython.display import IFrame

# Functions used

In [5]:
def load_scaler():
    """Loads in the required objects."""
    import pickle
    with open('scaler_song_recommender.pickle', 'rb') as f:
        return pickle.load(f)

In [6]:
def load_kmeans():
    """Loads in the required objects."""
    import pickle
    with open('kmeans_song_recommender.pickle', 'rb') as f:
        return pickle.load(f)

In [7]:
# writing the top_100 recommender function
def top_100_recommender():
    """Function that prompts the user for song input
    and checks whether it's in the top-100 csv file."""
    top_100 = pd.read_csv("top_100.csv")
    user_input = input("Please type in a song you like: ")
    if user_input.lower() in map(str.lower, list(top_100["Song"])):
        print("Your song is in the current top-100.")
        random_song = random.choice(list(top_100["Song"]))
        artist = top_100.loc[top_100["Song"] == random_song, 'Artist'].to_string(index=False)
        print(f"\nHere is another song from the top-100 you might like: {random_song} by {artist}")
    else: 
        print("\nYour song is not in the current top-100.")
        play_song(random_song_recommender())
        # for some reason the embedded track player doesn't show when this part is executed, even though the random_song_recommender() returns a valid song id

In [8]:
def random_song_recommender():
    """Prompts the use to input a song they like and finds a similar one in the song_df"""
    scaler = load_scaler()
    kmeans = load_kmeans()
    scaled_songs_df = pd.read_csv("scaled_songs_df.csv")
    user_song = input("Please give me the name of your favorite song, so that I recommend you another one you might like: ")
    result = sp.search(q=user_song,limit=1)
    result["tracks"]["items"][0]["id"]
    user_song_audio_features = sp.audio_features(result["tracks"]["items"][0]["id"])
    user_song_df = pd.DataFrame(user_song_audio_features)    
    user_song_df = user_song_df[["danceability","energy","loudness","speechiness","acousticness",
    "instrumentalness","liveness","valence","tempo","id","duration_ms"]]
    user_song_audio_features = user_song_df[["danceability", "energy", "loudness", "speechiness", "acousticness", "liveness", "valence", "tempo"]]
    user_song_audio_features_scaled = scaler.transform(user_song_audio_features)
    label = kmeans.predict(user_song_audio_features_scaled)
    label = int(label)
    scaled_user_song_df = pd.DataFrame(user_song_audio_features_scaled, columns=["danceability", "energy", "loudness", "speechiness", "acousticness", "liveness", "valence", "tempo"])
    scaled_user_song_df = scaled_user_song_df.join(user_song_df[["id", "duration_ms"]])
    sample_df = scaled_songs_df.loc[scaled_songs_df["clusters"]== label]
    rec_song_df = sample_df.sample(1)
    rec_song_id = rec_song_df["id"].to_string(index=False)
    return rec_song_id

In [9]:
def play_song(rec_song_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+rec_song_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

# Actual programme

In [10]:
# writing the loop that ensures the programme keeps running

while True:
    top_100_recommender()
    user_input = input("\nDo you want to input another song? yes/no: ")
    if user_input == "yes":
        continue
    elif user_input == "no":
        print("Your session has ended. Thank you for using our app!")
        break
    else:
        print("Please write either 'yes' or 'no'")
        continue

KeyboardInterrupt: Interrupted by user